#### P0P1AP2P3BP4P7DP6P11E

In [ ]:
# l - line follow        # a - turn anticlockwise
# c - turn clockwise     # f - move forward (for jumping nodes)

# A - time from P1 to A     # a - time from P2 to A
# B - time from P4 to B     # b - time from P3 to B
# C - time from P7 to C     # c - time from P8 to C
# D - time from P6 to D     # d - time from P7 to D
# E - time from P11 to E    # e - time from P9 to E

# orientation_vector = 0 --> north (bot is facing e-Yantra logo)
# orientation_vector = 1 --> east
# orientation_vector = 2 --> south
# orientation_vector = 3 --> west

# z - buzzer
# s - stop


##### Path Generator

In [4]:
import numpy as np

map = np.array(
    [
        [0, None, None, None, None, None],
        [None, 1, 5, 6, 11, None],  #
        [None, 21, None, 24, None, 25],  #
        [None, 2, 4, 7, 10, None],  #
        [None, None, 22, 23, None, None],  #
        [None, None, 3, 8, 9, None],
    ]
)


def map_rotate(dir):
    global map
    if dir == 1:  # Clockwise
        map = np.flip(map.transpose(), 0)
    elif dir == -1:  # AntiClockwise
        map = np.flip(map.transpose(), 1)
    # print(map)


def mov_gen(path_str):
    global map
    orientation_vector = 0
    moves = ""
    path_counter = 0  # pointer for shortest_path string
    nodes_list = []  # list for contating all the nodes for each event visit
    nodes = []
    for i in range(0, len(path_str)):
        if path_str[i] == "P":
            node = int(path_str[path_counter + 1])
            if path_str[path_counter + 2] == "0" or path_str[path_counter + 2] == "1":
                node += int(path_str[path_counter + 2]) + 9
            nodes.append(node)
        elif path_str[i] == "A":
            nodes.append(21)
        elif path_str[i] == "B":
            nodes.append(22)
        elif path_str[i] == "C":
            nodes.append(23)
        elif path_str[i] == "D":
            nodes.append(24)
        elif path_str[i] == "E":
            nodes.append(25)
        elif path_str[i] == "s":
            nodes_list = nodes
            nodes = []

        path_counter += 1

    print(nodes_list)

    line_follow_time = 0
    forward_time = 0
    backward_time = 0
    right_time = 0
    left_time = 0

    for i in range(0, len(nodes_list) - 1):
        node1 = nodes_list[i]
        node2 = nodes_list[i + 1]
        node0 = 0
        if i > 0:
            node0 = nodes_list[i - 1]
        print(str(node1) + " --> " + str(node2) + " : ", end="")
        pos_num1 = np.argwhere(map == node1)
        pos_num2 = np.argwhere(map == node2)
        # print("\npos_num1", pos_num1)
        # print("pos_num2", pos_num2)
        if node1 in [21, 22, 23, 24, 25]:
            if node0 in [1, 4, 7, 6, 11] and [node0, node1] != [7, 24]:
                moves += str(chr(64 + (node1 - 20))) + "z"
            else:
                moves += str(chr(96 + (node1 - 20))) + "z"
        else:
            moves += "l" + str(line_follow_time)

        if (
            abs(pos_num1[0][0] - pos_num2[0][0]) <= 2
            and abs(pos_num1[0][1] - pos_num2[0][1]) <= 2
            and (pos_num1[0][0] == pos_num2[0][0] or pos_num1[0][1] == pos_num2[0][1])
            and [node1, node2] != [0, 1]
        ):
            if (
                pos_num1[0][0] + 1 == pos_num2[0][0]
                or pos_num1[0][0] + 2 == pos_num2[0][0]
            ):
                print("Right")
                moves += "r" + str(right_time)
                map_rotate(1)
            elif (
                pos_num1[0][0] == pos_num2[0][0] + 1
                or pos_num1[0][0] == pos_num2[0][0] + 2
            ):
                print("Left")
                moves += "a" + str(left_time)
                map_rotate(-1)
            elif (
                pos_num1[0][1] + 1 == pos_num2[0][1]
                or pos_num1[0][1] + 2 == pos_num2[0][1]
            ):
                print("forward")
                moves += "f" + str(forward_time)
            elif (
                pos_num1[0][1] == pos_num2[0][1] + 1
                or pos_num1[0][1] == pos_num2[0][1] + 2
            ):
                print("U-Turn")
                if moves[-2:-1] in ['A',"B","C","D","E"]:
                    moves += "u" + str(1)
                else:
                    moves += "u" + str(0)               
                map_rotate(1)
                map_rotate(1)
            else:
                print("none1")

        elif [node0, node1, node2] == [6, 11, 25]:
            print("Forward")
            moves += "f" + str(forward_time)
            map_rotate(1)
        elif [node0, node1, node2] == [10, 11, 25]:
            print("Right")
            moves += "r" + str(forward_time)
            map_rotate(1)
            map_rotate(1)
        elif [node1, node2] == [25, 11]:
            print("U-turn")
            moves += "u" + str(forward_time)
            map_rotate(1)
        elif [node0, node1, node2] == [2, 1, 0]:
            print("Left")
            moves += "a" + str(forward_time)
            map_rotate(-1)
        elif [node0, node1, node2] == [5, 1, 0]:
            print("Forward")
            moves += "f" + str(forward_time)
        elif [node1, node2] == [0, 1]:
            print("Forward")
            moves = moves[:-2]
        else:
            print("none2")

    moves += "ss"
    return moves


moves = mov_gen("P0P1P5P6P11EP11P10P9P8P3BP4P2AP1P5P6DP7CP7P4P2P1P0s")
print("moves", moves)
# expected result: l0c0l0f0l0a0l0c0l0a0l0c0l0fE
#   l0f0l0f0l0f0l0f0Ezu0l0a0l0f0l0r0l0f0l0r0bzf0l0a0l0r0azf0l0r0l0f0l0r0Dzf0l0f0Czu1l0a0l0f0l0r0l0a0ss


[0, 1, 5, 6, 11, 25, 11, 10, 9, 8, 3, 22, 4, 2, 21, 1, 5, 6, 24, 7, 23, 7, 4, 2, 1, 0]
0 --> 1 : Forward
1 --> 5 : forward
5 --> 6 : forward
6 --> 11 : forward
11 --> 25 : Forward
25 --> 11 : U-turn
11 --> 10 : Left
10 --> 9 : forward
9 --> 8 : Right
8 --> 3 : forward
3 --> 22 : Right
22 --> 4 : forward
4 --> 2 : Left
2 --> 21 : Right
21 --> 1 : forward
1 --> 5 : Right
5 --> 6 : forward
6 --> 24 : Right
24 --> 7 : forward
7 --> 23 : forward
23 --> 7 : U-Turn
7 --> 4 : Left
4 --> 2 : forward
2 --> 1 : Right
1 --> 0 : Left
moves l0f0l0f0l0f0l0f0Ezu0l0a0l0f0l0r0l0f0l0r0bzf0l0a0l0r0azf0l0r0l0f0l0r0Dzf0l0f0Czu1l0a0l0f0l0r0l0a0ss


##### WiFi Connect

In [ ]:
import socket

# ESP32 IP address
esp32_ip = "192.168.1.6"  # Replace with the actual IP address of your ESP32
client_socket = socket.socket(
    socket.AF_INET, socket.SOCK_STREAM
)  # Create a socket object
server_address = (esp32_ip, 80)  # Connect to the ESP32 server
client_socket.connect(server_address)

message = mov_gen("P4P3P2zs")
print(message)

client_socket.sendall(message.encode())
data = client_socket.recv(1024)  # Receive the response
print(data.decode())
client_socket.close()  # Close the socket


In [4]:
import cv2 as cv
import numpy as np




##### Dijkstra's Algorithm

In [ ]:
# Eyrc node-node measurements

# 1. P0-P1 - 27cm
# 2. P1-P2 -97cm
# 3. P1-P5-50cm
# 4. P1-A -35cm
# 5. P2-P4- 52cm
# 6. P2-P3 -160cm
# 7. P3-P4 - 103cm
# 8. P3-B -45cm
# 9. P4-B - 60cm
# 10. P4-P7-51cm
# 11. P4-P5 -100cm
# 12. P5-P6- 57 cm
# 13. P6-P11- 40cm
# 14. P6-P7 - 100cm
# 15. P6- D - 38cm
# 16. P7-D -62 cm
# 17. P7-P10- 38cm
# 18. P7-P8 -103cm
# 19. P7-C - 65 cm
# 20. P8- C - 41 cm
# 21. P8-P3 - 50cm
# 22. P8-P9 -38cm
# 23. P9-P10 - 102cm
# 24. P10-P11- 101cm
# 25. P11-E -71 cm
# 26. P9- E - 186cm

# path_distances = [21, 246, 655, 26, 241, 246, 474, 12, 4, 13, 0, 461, 432]
# import heapq


# def dijkstra(graph, start):
#     distances = {node: float("infinity") for node in graph}
#     distances[start] = 0
#     queue = [(0, start)]

#     while queue:
#         current_distance, current_node = heapq.heappop(queue)

#         if current_distance > distances[current_node]:
#             continue

#         for neighbor, weight in graph[current_node].items():
#             distance = current_distance + weight

#             if distance < distances[neighbor]:
#                 distances[neighbor] = distance
#                 heapq.heappush(queue, (distance, neighbor))

#     return distances


# # Example usage:
# graph = {
#     "A": {"B": 1, "C": 3},
#     "B": {"A": 1, "C": 2, "D": 4},
#     "C": {"A": 3, "B": 2, "D": 1},
#     "D": {"B": 4, "C": 1},
# }
# print(dijkstra(graph, "A"))  # {'A': 0, 'B': 1, 'C': 2, 'D': 3}

# import numpy as np


# def find_relative_position(array, number1, number2):
#     pos_num1 = np.argwhere(array == number1)
#     pos_num2 = np.argwhere(array == number2)

#     if pos_num1.size == 0 or pos_num2.size == 0:
#         return "One or both numbers not found in the array."

#     if pos_num1[0][0] < pos_num2[0][0]:
#         return f"Number {number1} is above Number {number2}"
#     elif pos_num1[0][0] > pos_num2[0][0]:
#         return f"Number {number1} is below Number {number2}"
#     elif pos_num1[0][1] < pos_num2[0][1]:
#         return f"Number {number1} is to the left of Number {number2}"
#     elif pos_num1[0][1] > pos_num2[0][1]:
#         return f"Number {number1} is to the right of Number {number2}"
#     else:
#         return "Both numbers are at the same position."


# array = np.array(
#     [
#         [0, 1, 5, 6, 11],
#         [None, 2, 4, 7, 10],
#         [None, None, 3, 8, 9],
#     ]
# )

# print(find_relative_position(array, 2, 1))
